In [149]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict,Annotated
from pydantic import BaseModel,Field
from dotenv import load_dotenv
load_dotenv()
from langchain_google_genai import ChatGoogleGenerativeAI
import operator

In [150]:
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")  

In [ ]:
class Evaluations_schema(BaseModel):
    feedback: str = Field(description='Detailed feedback for the essay')
    score: int = Field(description='Score out of 10', ge=0, le=10)

In [152]:
Structuredmodel = model.with_structured_output(Evaluations_schema)

In [ ]:
essay ="""The world today stands at a crossroads where opportunities and challenges exist side by side. In almost every sphere of life—politics, economy, technology, environment, and society—the current situation reflects a complex interplay of progress and crisis. Understanding these conditions helps us not only recognize where humanity stands today but also imagine the path we should take in the future.On a global scale, the economy is one of the most pressing concerns. Many countries are facing high inflation, rising living costs, and widening gaps between the rich and the poor. The aftershocks of the COVID-19 pandemic, coupled with ongoing conflicts such as the war in Ukraine and tensions in other parts of the world, have disrupted global trade and energy supplies. Alongside economic troubles, climate change is creating severe challenges. Unpredictable weather patterns, heat waves, floods, and wildfires are becoming frequent reminders that environmental action cannot be delayed. In contrast, rapid technological development—especially artificial intelligence and digital innovation—has opened new doors for education, health, and communication, though it has also raised ethical and employment concerns.Looking at Nepal, the country mirrors both global and local challenges. Economic uncertainty is visible in rising unemployment, dependence on remittances, and the slow pace of industrial growth. Political instability continues to influence policy-making and weaken trust in governance. On the other hand, Nepal holds immense potential in tourism, hydropower, and agriculture. The increasing use of digital platforms by the younger generation is another sign of positive change. However, issues such as migration of skilled youth, poor infrastructure, and vulnerability to natural disasters remain as obstacles. The current situation in Nepal is thus a mixture of hardship and hope.

In a more general sense, the current situations worldwide highlight the contradictions of modern life. While science and technology bring people closer, misinformation and polarization divide societies. While economic growth creates wealth, unequal distribution fuels dissatisfaction. While humans have more knowledge than ever before, the planet suffers from reckless exploitation of resources. These contradictions call for balance—between progress and sustainability, between development and fairness, and between national interest and global responsibility.

In conclusion, the present era is both an age of uncertainty and an age of possibility. Globally, we face economic, environmental, and political challenges; nationally, Nepal reflects many of these struggles in its own form; and generally, the world must learn to balance innovation with compassion. The choices made today will shape not only the destiny of countries like Nepal but also the entire human civilization. It is therefore the responsibility of individuals, communities, and leaders alike to turn current challenges into opportunities for a more stable and just future."""

In [154]:
result = Structuredmodel.invoke(essay)

In [155]:
result.feedback

'This is a well-written essay that effectively summarizes the current global and local challenges and opportunities. The introduction clearly sets the stage, and the essay maintains a consistent focus on the interplay between progress and crisis. The use of examples, both global and specific to Nepal, strengthens the arguments. The conclusion is strong and emphasizes the importance of responsible decision-making. However, the essay could benefit from more in-depth analysis and specific recommendations for addressing the challenges identified.'

In [156]:
result.score

7

In [157]:
class essay_state(TypedDict):
    essay: str
    language_feedback: str
    analysis_feedback: str
    clarity_feedback: str
    overall_feedback: str
    individual_scores: Annotated[list[int], operator.add]
    avg_score: float

In [ ]:
def evaluate_languages(state: essay_state):
    essay = state["essay"]
    prompt = f"""Evaluate the language quality of the following essay. Provide detailed feedback including strengths and areas for improvement, such as grammar, vocabulary, sentence structure, and overall readability. Then, assign a score out of 10 based on the quality.

Essay:
{essay}"""
    output = Structuredmodel.invoke(prompt)
    return{"language_feedback": output.feedback,"individual_scores":[output.score]}

In [ ]:
def evaluate_analysis(state: essay_state):
    essay = state["essay"]
    prompt = f"""Evaluate the depth of analysis in the following essay. Provide detailed feedback on how well the essay analyzes the topic, including the use of evidence, logical reasoning, and critical thinking. Identify strengths and weaknesses. Then, assign a score out of 10.

Essay:
{essay}"""
    output = Structuredmodel.invoke(prompt)
    return{"analysis_feedback":output.feedback,"individual_scores":[output.score]}

In [ ]:
def evaluate_thought(state: essay_state):
    essay = state["essay"]
    prompt = f"""Evaluate the clarity of thought in the following essay. Provide detailed feedback on the organization of ideas, logical flow, coherence, and how clearly the main arguments are presented. Include strengths and suggestions for improvement. Then, assign a score out of 10.

Essay:
{essay}"""
    output = Structuredmodel.invoke(prompt)
    return{"clarity_feedback":output.feedback,"individual_scores":[output.score]}

In [161]:

def final_evaluation(state: essay_state):

    # summary feedback
    prompt = f'Based on the following feedbacks create a summarized feedback \n language feedback - {state["language_feedback"]} \n depth of analysis feedback - {state["analysis_feedback"]} \n clarity of thought feedback - {state["clarity_feedback"]}'
    overall_feedback = model.invoke(prompt).content

    # avg calculate
    avg_score = sum(state['individual_scores'])/len(state['individual_scores'])

    return {'overall_feedback': overall_feedback, 'avg_score': avg_score}

In [162]:
graph = StateGraph(essay_state)
graph.add_node("evaluate_languages",evaluate_languages)
graph.add_node("evaluate_analysis",evaluate_analysis)
graph.add_node("evaluate_thought",evaluate_thought)
graph.add_node("final_evaluation",final_evaluation)


graph.add_edge(START, 'evaluate_languages')
graph.add_edge(START, 'evaluate_analysis')
graph.add_edge(START, 'evaluate_thought')

graph.add_edge('evaluate_languages', 'final_evaluation')
graph.add_edge('evaluate_analysis', 'final_evaluation')
graph.add_edge('evaluate_thought', 'final_evaluation')

graph.add_edge('final_evaluation', END)

workflow = graph.compile()



In [163]:
essay2 = """  
### Essay on the Current Situation of Nepal

Nepal today is passing through a period of transition where challenges and opportunities are deeply interconnected. The country, known for its natural beauty and cultural diversity, is facing social, economic, and political issues that shape the lives of its people in significant ways. At the same time, Nepal also holds great potential for growth if these challenges are addressed with vision and responsibility.

Economically, Nepal struggles with unemployment and dependency on foreign remittances. A large number of young people are leaving the country in search of jobs abroad, which has created a gap in the domestic workforce. Although remittances contribute heavily to the national economy, overreliance on them prevents long-term economic stability. Inflation and rising prices of daily goods have further added pressure on ordinary citizens. At the same time, Nepal has untapped potential in tourism, agriculture, and hydropower. If developed properly, these sectors can strengthen the economy and provide opportunities for the youth inside the country.

Politically, Nepal continues to experience instability. Frequent changes in government, lack of effective policy implementation, and weak governance have created frustration among the people. Citizens often feel that leaders prioritize personal and party interests over national development. Despite having a democratic system, the gap between promises and delivery has weakened public trust in politics.

Socially, Nepal faces issues such as poverty, inequality, and limited access to quality education and healthcare. Rural areas still lack proper infrastructure like roads, schools, and hospitals, which makes development uneven across the country. Natural disasters such as floods and landslides add to the difficulties faced by vulnerable communities. However, there is also progress—an increasing number of youths are using digital technology, entrepreneurship is slowly growing, and awareness of social issues is spreading.

In terms of opportunities, Nepal stands at a unique position. Its natural resources, especially rivers, can be a major source of clean hydropower energy not only for domestic use but also for export to neighboring countries. Tourism remains another powerful sector, with Nepal’s mountains, culture, and heritage attracting global attention. Moreover, the energy, creativity, and determination of the younger generation can drive innovation and reform in the coming years.

In conclusion, the current situation of Nepal is a blend of struggle and hope. Economic dependence, political instability, and social challenges weigh heavily on the country, yet the opportunities in natural resources, tourism, and youth potential show a bright path forward. The future of Nepal depends on wise leadership, active citizen participation, and the ability to transform challenges into opportunities. If the present issues are addressed with seriousness, Nepal can move toward stability, prosperity, and a stronger global identity.


      """

In [ ]:
initial_state = {
    'essay': essay
}

workflow.invoke(initial_state)

{'essay': 'The world today stands at a crossroads where opportunities and challenges exist side by side. In almost every sphere of life—politics, economy, technology, environment, and society—the current situation reflects a complex interplay of progress and crisis. Understanding these conditions helps us not only recognize where humanity stands today but also imagine the path we should take in the future.On a global scale, the economy is one of the most pressing concerns. Many countries are facing high inflation, rising living costs, and widening gaps between the rich and the poor. The aftershocks of the COVID-19 pandemic, coupled with ongoing conflicts such as the war in Ukraine and tensions in other parts of the world, have disrupted global trade and energy supplies. Alongside economic troubles, climate change is creating severe challenges. Unpredictable weather patterns, heat waves, floods, and wildfires are becoming frequent reminders that environmental action cannot be delayed. I